# Initial SQL Queries
Here I'm hoping to make a baseline SQL query that I can use to make my first model

Minimum requirements:
 - Selects the sale price from the `sales` table
 - Selects at least one other attribute from the `sales` table
 - Joins on the `buildings` table and selects at least one attribute
 - Joins on the `parcels` table and selects at least one attribute
 - Filters to only include home sales from 2018

In [1]:
import psycopg2
import pandas as pd

In [2]:
conn = psycopg2.connect(dbname="housing_data")

In [4]:
pd.read_sql_query("SELECT * FROM sales LIMIT 5;", conn)

,excisetaxnbr,major,minor,documentdate,saleprice,recordingnbr,volume,page,platnbr,plattype,...,propertytype,principaluse,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning
0,1600768,330405,0100,1998-03-19,215000,199803251689,145,039,330405,C,...,2,2,3,N,N,N,N,1,3,
1,2413752,868146,0030,2009-09-11,0,20091022001461,,,,,...,3,2,15,N,N,N,N,11,3,18 31 38
2,1939480,258190,0265,2003-02-06,0,20030214003390,,,,,...,3,6,15,N,N,N,N,10,8,31 51
3,2999169,919715,0200,2019-07-08,192000,20190712001080,,,,,...,3,2,3,N,N,N,N,1,3,
4,2220242,334330,1343,2006-05-30,0,20060706002163,,,,,...,1,6,3,N,N,N,N,18,7,11 31


In [7]:
q1 = """
    SELECT SalePrice
    FROM sales
    LIMIT 10;
"""

In [8]:
pd.read_sql_query(q1, conn)

,saleprice
0,215000
1,0
2,0
3,192000
4,0
5,690576
6,2340000
7,2340000
8,2340000
9,2340000


In [9]:
q2 = """
    SELECT SalePrice, AFHistoricProperty
    FROM sales
    LIMIT 10;
"""

In [10]:
pd.read_sql_query(q2, conn)

,saleprice,afhistoricproperty
0,215000,N
1,0,N
2,0,N
3,192000,N
4,0,N
5,690576,N
6,2340000,N
7,2340000,N
8,2340000,N
9,2340000,N


In [13]:
q3 = """
    SELECT sales.SalePrice, sales.AFHistoricProperty::boolean
    FROM sales
    LIMIT 10;
"""

In [14]:
pd.read_sql_query(q3, conn)

,saleprice,afhistoricproperty
0,215000,False
1,0,False
2,0,False
3,192000,False
4,0,False
5,690576,False
6,2340000,False
7,2340000,False
8,2340000,False
9,2340000,False


In [19]:
q4 = """
    SELECT
        sales.SalePrice, 
        sales.AFHistoricProperty::boolean,
        parcels.WfntLocation
    FROM sales
    INNER JOIN parcels
    ON (parcels.Major = sales.Major AND parcels.Minor = sales.Minor)
    WHERE sales.AFHistoricProperty != ' '
    LIMIT 10;
"""

In [20]:
pd.read_sql_query(q4, conn)

,saleprice,afhistoricproperty,wfntlocation
0,2340000,False,0
1,5400000,False,0
2,0,False,0
3,5518000,False,0
4,5518000,False,0
5,5518000,False,0
6,589950,False,0
7,2800000,False,0
8,0,False,0
9,13000,False,8


Noting here: I am ignoring all rows where Historic Property field is malformed, because I'm using the Historic Property feature as my sole feature from the `sales` table.  In the future I may change this 

In [21]:
q5 = """
    SELECT
        sales.SalePrice, 
        sales.AFHistoricProperty::boolean,
        parcels.WfntLocation,
        buildings.SqFtTotLiving
    FROM sales
    INNER JOIN parcels
        ON (parcels.Major = sales.Major AND parcels.Minor = sales.Minor)
    INNER JOIN buildings
        ON (buildings.Major = parcels.Major AND buildings.Minor = parcels.Minor)
    WHERE sales.AFHistoricProperty != ' '
    LIMIT 10;
"""

In [22]:
pd.read_sql_query(q5, conn)

,saleprice,afhistoricproperty,wfntlocation,sqfttotliving
0,285000,False,0,1710
1,0,False,0,1510
2,670000,False,0,2420
3,113000,False,0,1580
4,0,False,0,1150
5,175000,False,0,2700
6,475000,False,0,1560
7,239500,False,0,1870
8,580000,False,0,1520
9,255000,False,0,1540


The documentation for the Waterfront Location attribute says that there are codes 1-9 indicating the specific waterfronts (Elliott Bay, Lake Washington, etc.) but I just want it to be a boolean indicating True if the code is 1-9 or False if the code is 0

In [37]:
q6 = """
    SELECT
        CONCAT(sales.Major, sales.Minor) AS PIN,     -- parcel id number
        sales.SalePrice, 
        sales.AFHistoricProperty::boolean,           -- cast Y or N to boolean
        sales.DocumentDate,
        CASE
            WHEN parcels.WfntLocation > 0            -- 1-9 indicate particular bodies of water
                THEN TRUE
            ELSE                                     -- I infer that 0 means no waterfront
                FALSE
        END as WfntLocation,
        buildings.SqFtTotLiving
    FROM sales                                       -- start the join with sales bc sale price is target
    INNER JOIN parcels ON (                          -- parcel major + minor is the unique identifier
        parcels.Major = sales.Major                  -- (parcels are the things being sold in the sales)
        AND parcels.Minor = sales.Minor
    )
    INNER JOIN buildings ON (                        -- building belongs to one parcel
        buildings.Major = parcels.Major              -- parcel can have many buildings (unclear how often)
        AND buildings.Minor = parcels.Minor
    )
    WHERE (
        date_part('year', sales.DocumentDate) = 2018 -- 2018 is the specified year
        AND sales.AFHistoricProperty != ' '          -- don't include malformed historic property info
        AND sales.SalePrice > 0                      -- assume that sale price of 0 is bad data
    )
    ORDER BY sales.DocumentDate;
"""

In [38]:
sales_df = pd.read_sql_query(q6, conn)

In [39]:
sales_df.describe()

,saleprice,sqfttotliving
count,3.027000e+04,30270.000000
mean,8.240762e+05,2176.427288
std,7.799831e+05,1007.658084
min,1.000000e+00,3.000000
25%,4.405000e+05,1450.000000
50%,6.600000e+05,1980.000000
75%,9.360000e+05,2720.000000
max,2.675000e+07,14980.000000


In [40]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30270 entries, 0 to 30269
Data columns (total 6 columns):
pin                   30270 non-null object
saleprice             30270 non-null int64
afhistoricproperty    30270 non-null bool
documentdate          30270 non-null object
wfntlocation          30270 non-null bool
sqfttotliving         30270 non-null int64
dtypes: bool(2), int64(2), object(2)
memory usage: 1005.2+ KB


In [41]:
sales_df["afhistoricproperty"].value_counts()

False    30268
True         2
Name: afhistoricproperty, dtype: int64

In [42]:
sales_df["wfntlocation"].value_counts()

False    29725
True       545
Name: wfntlocation, dtype: int64

Eh, maybe we can re-add the Historic Property attribute later.  For now it's cluttering up the query and I'm skeptical about the data when there are only 2 out of 30k

In [43]:
q7 = """
    SELECT
        CONCAT(sales.Major, sales.Minor) AS PIN,     -- parcel id number
        sales.SalePrice,
        sales.DocumentDate,
        CASE
            WHEN parcels.WfntLocation > 0            -- 1-9 indicate particular bodies of water
                THEN TRUE
            ELSE                                     -- I infer that 0 means no waterfront
                FALSE
        END as WfntLocation,
        buildings.SqFtTotLiving
    FROM sales                                       -- start the join with sales bc sale price is target
    INNER JOIN parcels ON (                          -- parcel major + minor is the unique identifier
        parcels.Major = sales.Major                  -- (parcels are the things being sold in the sales)
        AND parcels.Minor = sales.Minor
    )
    INNER JOIN buildings ON (                        -- building belongs to one parcel
        buildings.Major = parcels.Major              -- parcel can have many buildings (unclear how often)
        AND buildings.Minor = parcels.Minor
    )
    WHERE (
        date_part('year', sales.DocumentDate) = 2018 -- 2018 is the specified year
        AND sales.SalePrice > 0                      -- assume that sale price of 0 is bad data
    )
    ORDER BY sales.DocumentDate;
"""

In [44]:
sales_df = pd.read_sql_query(q7, conn)

In [45]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30270 entries, 0 to 30269
Data columns (total 5 columns):
pin              30270 non-null object
saleprice        30270 non-null int64
documentdate     30270 non-null object
wfntlocation     30270 non-null bool
sqfttotliving    30270 non-null int64
dtypes: bool(1), int64(2), object(2)
memory usage: 975.6+ KB


In [46]:
conn.close()

Double-checking that it works in the file

In [47]:
from sql_utils import create_sales_df
sales_df = create_sales_df()

In [48]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30270 entries, 0 to 30269
Data columns (total 5 columns):
pin              30270 non-null object
saleprice        30270 non-null int64
documentdate     30270 non-null object
wfntlocation     30270 non-null bool
sqfttotliving    30270 non-null int64
dtypes: bool(1), int64(2), object(2)
memory usage: 975.6+ KB


In [49]:
sales_df.head()

,pin,saleprice,documentdate,wfntlocation,sqfttotliving
0,8732160190,355000,2018-01-01,False,1580
1,2287300010,298633,2018-01-01,False,1810
2,8695200067,275000,2018-01-01,False,1250
3,5569610110,429950,2018-01-02,False,2140
4,1594600015,1199700,2018-01-02,False,2870
